In [378]:
%load_ext autoreload

%autoreload 2

import configparser
import os
import time
import random
import pandas as pd
import requests
import openpyxl
import sqlite3

from datetime import datetime
from io import StringIO
from pathlib import Path
from bs4 import BeautifulSoup
from alive_progress import alive_bar

# some stuff I set up in a config file so I don't have to keep updating certain
# variables in every script
config = configparser.ConfigParser()
config.read('../src/config.ini')
output = Path(config['paths']['output'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [399]:
players_24 = pd.read_csv(config['data']['all_player_data_2024'])
players_23 = pd.read_csv(config['data']['all_player_data_2023'])
players_22 = pd.read_csv(config['data']['all_player_data_2022'])

In [380]:
players_24.head()

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,...,Passes,Passes.1,Passes.2,Passes.3,Carries,Carries.1,Take-Ons,Take-Ons.1,Unnamed: 36_level_0,player
0,NaN,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,...,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ,Match Report,NaN
1,0.0,2023-06-16,Fri,UEFA Euro Qualifying,Group stage,Away,W 2–1,il Israel,by Belarus,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,https://fbref.com/en/players/f935709c/matchlog...
2,1.0,2023-06-19,Mon,UEFA Euro Qualifying,Group stage,Home,W 2–1,il Israel,ad Andorra,N,...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,https://fbref.com/en/players/f935709c/matchlog...
3,2.0,2024-01-02,Tue,Premiership,Regular season,Away,W 3–0,Celtic,St Mirren,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,https://fbref.com/en/players/f935709c/matchlog...
4,3.0,2024-01-27,Sat,Premiership,Regular season,Home,W 1–0,Celtic,Ross County,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,https://fbref.com/en/players/f935709c/matchlog...


In [381]:
players_23.head()

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,...,Passes,Passes.1,Passes.2,Passes.3,Carries,Carries.1,Take-Ons,Take-Ons.1,Unnamed: 36_level_0,player
0,NaN,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,...,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ,Match Report,NaN
1,0.0,2023-02-25,Sat,MLS,Regular Season,Away,L 2–3,Toronto FC,D.C. United,N,...,16,21,76.2,0,13,1,0,0,Match Report,https://fbref.com/en/players/214d2406/matchlog...
2,1.0,2023-03-04,Sat,MLS,Regular Season,Away,D 1–1,Toronto FC,Atlanta Utd,N,...,7,8,87.5,0,6,0,0,0,Match Report,https://fbref.com/en/players/214d2406/matchlog...
3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://fbref.com/en/players/214d2406/matchlog...
4,3.0,2023-03-18,Sat,MLS,Regular Season,Home,W 2–0,Toronto FC,Inter Miami,N,...,4,5,80.0,0,12,1,2,0,Match Report,https://fbref.com/en/players/214d2406/matchlog...


In [382]:
players_22.head()

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,...,Passes,Passes.1,Passes.2,Passes.3,Carries,Carries.1,Take-Ons,Take-Ons.1,Unnamed: 36_level_0,player
0,NaN,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,...,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ,Match Report,NaN
1,0.0,2022-02-26,Sat,MLS,Regular Season,Home,D 1–1,Philadelphia Union,Minnesota Utd,N,...,6,7,85.7,2,6,1,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
2,1.0,2022-03-05,Sat,MLS,Regular Season,Away,W 2–1,Philadelphia Union,CF Montréal,N,...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,https://fbref.com/en/players/672f4300/matchlog...
3,2.0,2022-03-12,Sat,MLS,Regular Season,Home,W 2–0,Philadelphia Union,SJ Earthquakes,N,...,4,4,100.0,1,7,1,1,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
4,3.0,2022-03-19,Sat,MLS,Regular Season,Away,W 2–0,Philadelphia Union,NYCFC,N,...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,https://fbref.com/en/players/672f4300/matchlog...


In [400]:
players_24.columns = players_24.iloc[0]
players_24 = players_24.iloc[1:,1:]
players_23.columns = players_23.iloc[0]
players_23 = players_23.iloc[1:,1:]
players_22.columns = players_22.iloc[0]
players_22 = players_22.iloc[1:,1:]


In [401]:
players_24.shape[0] + players_23.shape[0] + players_22.shape[0]

77796

In [384]:
correct_col_names = ['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA',
       'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att_TakeOn', 'Succ',
       'Match Report', 'player_url']

In [385]:
players_24.columns = correct_col_names
players_23.columns = correct_col_names
players_22.columns = correct_col_names

In [386]:
players_24 = players_24[players_24['Comp'] == 'MLS']
players_23 = players_23[players_23['Comp'] == 'MLS']
players_22 = players_22[players_22['Comp'] == 'MLS']

In [387]:
all_players = pd.concat([players_24, players_23, players_22], ignore_index=True)

In [388]:
all_players

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att_TakeOn,Succ,Match Report,player_url
0,2024-03-30,Sat,MLS,Regular Season,Home,D 1–1,Charlotte,FC Cincinnati,N,RW,...,4,6,66.7,0,9,3,0,0,Match Report,https://fbref.com/en/players/f935709c/matchlog...
1,2024-04-06,Sat,MLS,Regular Season,Away,L 0–1,Charlotte,NE Revolution,Y,RW,...,16,20,80.0,0,17,2,1,1,Match Report,https://fbref.com/en/players/f935709c/matchlog...
2,2024-04-13,Sat,MLS,Regular Season,Home,W 3–2,Charlotte,Toronto FC,Y,RW,...,12,18,66.7,6,17,5,4,2,Match Report,https://fbref.com/en/players/f935709c/matchlog...
3,2024-04-21,Sun,MLS,Regular Season,Home,L 0–3,Charlotte,Minnesota Utd,Y,RW,...,16,36,44.4,1,25,3,3,0,Match Report,https://fbref.com/en/players/f935709c/matchlog...
4,2024-04-27,Sat,MLS,Regular Season,Away,L 1–2,Charlotte,NYCFC,Y,RW,...,11,14,78.6,2,16,1,3,0,Match Report,https://fbref.com/en/players/f935709c/matchlog...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59658,2022-09-27,Tue,MLS,Regular Season,Away,D 1–1,FC Cincinnati,Seattle Sounders FC,Y,CB,...,27,31,87.1,1,22,0,1,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
59659,2022-10-01,Sat,MLS,Regular Season,Home,L 2–3,FC Cincinnati,Chicago Fire,Y,CB,...,30,38,78.9,4,16,0,1,1,Match Report,https://fbref.com/en/players/672f4300/matchlog...
59660,2022-10-09,Sun,MLS,Regular Season,Away,W 5–2,FC Cincinnati,D.C. United,Y,CB,...,16,21,76.2,2,15,0,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
59661,2022-10-15,Sat,MLS,First Round,Away,W 2–1,FC Cincinnati,NY Red Bulls,Y,CB,...,10,15,66.7,1,5,0,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...


In [389]:
game_keys = []
for index, player in all_players.iterrows():
    if player['Venue'] == 'Home':
        game_keys.append(f'{player['Date']} {player['Squad']} vs {player['Opponent']}')
    else:
        game_keys.append(f'{player['Date']} {player['Opponent']} vs {player['Squad']}')

In [390]:
all_players['game_key'] = game_keys
all_players

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Att,Cmp%,PrgP,Carries,PrgC,Att_TakeOn,Succ,Match Report,player_url,game_key
0,2024-03-30,Sat,MLS,Regular Season,Home,D 1–1,Charlotte,FC Cincinnati,N,RW,...,6,66.7,0,9,3,0,0,Match Report,https://fbref.com/en/players/f935709c/matchlog...,2024-03-30 Charlotte vs FC Cincinnati
1,2024-04-06,Sat,MLS,Regular Season,Away,L 0–1,Charlotte,NE Revolution,Y,RW,...,20,80.0,0,17,2,1,1,Match Report,https://fbref.com/en/players/f935709c/matchlog...,2024-04-06 NE Revolution vs Charlotte
2,2024-04-13,Sat,MLS,Regular Season,Home,W 3–2,Charlotte,Toronto FC,Y,RW,...,18,66.7,6,17,5,4,2,Match Report,https://fbref.com/en/players/f935709c/matchlog...,2024-04-13 Charlotte vs Toronto FC
3,2024-04-21,Sun,MLS,Regular Season,Home,L 0–3,Charlotte,Minnesota Utd,Y,RW,...,36,44.4,1,25,3,3,0,Match Report,https://fbref.com/en/players/f935709c/matchlog...,2024-04-21 Charlotte vs Minnesota Utd
4,2024-04-27,Sat,MLS,Regular Season,Away,L 1–2,Charlotte,NYCFC,Y,RW,...,14,78.6,2,16,1,3,0,Match Report,https://fbref.com/en/players/f935709c/matchlog...,2024-04-27 NYCFC vs Charlotte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59658,2022-09-27,Tue,MLS,Regular Season,Away,D 1–1,FC Cincinnati,Seattle Sounders FC,Y,CB,...,31,87.1,1,22,0,1,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...,2022-09-27 Seattle Sounders FC vs FC Cincinnati
59659,2022-10-01,Sat,MLS,Regular Season,Home,L 2–3,FC Cincinnati,Chicago Fire,Y,CB,...,38,78.9,4,16,0,1,1,Match Report,https://fbref.com/en/players/672f4300/matchlog...,2022-10-01 FC Cincinnati vs Chicago Fire
59660,2022-10-09,Sun,MLS,Regular Season,Away,W 5–2,FC Cincinnati,D.C. United,Y,CB,...,21,76.2,2,15,0,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...,2022-10-09 D.C. United vs FC Cincinnati
59661,2022-10-15,Sat,MLS,First Round,Away,W 2–1,FC Cincinnati,NY Red Bulls,Y,CB,...,15,66.7,1,5,0,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...,2022-10-15 NY Red Bulls vs FC Cincinnati


In [391]:
all_matches = all_players.copy()[['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent']]
all_matches = all_matches[all_matches['Venue']=='Home']
all_matches['Results_Raw'] = all_matches['Result'].apply(lambda x: x.replace(' ', '–').replace('(', '').replace(')', '').split('–'))
all_matches['OverallResult'] = all_matches['Results_Raw'].apply(lambda x: x[0])
all_matches['home_score'] = all_matches['Results_Raw'].apply(lambda x: int(x[1]))
all_matches['home_penalties' ] = all_matches['Results_Raw'].apply(lambda x: int(x[2]) if len(x)==5 else 0)
all_matches['away_score'] = all_matches['Results_Raw'].apply(lambda x: int(x[3]) if len(x)==5 else int(x[2]))
all_matches['away_penalties' ] = all_matches['Results_Raw'].apply(lambda x: int(x[4]) if len(x)==5 else 0)
all_matches['game_key'] = all_matches.apply(lambda x: f'{x['Date']} {x['Squad']} vs {x['Opponent']}', axis=1)

In [392]:
all_matches = all_matches[['Date', 'Day', 'Round', 'Squad', 'Opponent', 'Result', 'OverallResult', 'home_score', 'home_penalties', 'away_score', 'away_penalties', 'game_key']]
all_matches.columns = ['game_date', 'day', 'round', 'home_team', 'away_team', 'result', 'overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties', 'game_key']

In [393]:
all_matches.drop_duplicates(inplace=True)

In [394]:
all_matches

,game_date,day,round,home_team,away_team,result,overall_result,home_score,home_penalties,away_score,away_penalties,game_key
0,2024-03-30,Sat,Regular Season,Charlotte,FC Cincinnati,D 1–1,D,1,0,1,0,2024-03-30 Charlotte vs FC Cincinnati
2,2024-04-13,Sat,Regular Season,Charlotte,Toronto FC,W 3–2,W,3,0,2,0,2024-04-13 Charlotte vs Toronto FC
3,2024-04-21,Sun,Regular Season,Charlotte,Minnesota Utd,L 0–3,L,0,0,3,0,2024-04-21 Charlotte vs Minnesota Utd
5,2024-05-25,Sat,Regular Season,Charlotte,Philadelphia Union,D 0–0,D,0,0,0,0,2024-05-25 Charlotte vs Philadelphia Union
8,2024-06-15,Sat,Regular Season,Charlotte,D.C. United,W 1–0,W,1,0,0,0,2024-06-15 Charlotte vs D.C. United
...,...,...,...,...,...,...,...,...,...,...,...,...
44923,2022-05-28,Sat,Regular Season,Sporting KC,Vancouver W'caps,L 0–1,L,0,0,1,0,2022-05-28 Sporting KC vs Vancouver W'caps
44926,2022-07-03,Sun,Regular Season,Sporting KC,NY Red Bulls,L 0–1,L,0,0,1,0,2022-07-03 Sporting KC vs NY Red Bulls
45417,2022-04-09,Sat,Regular Season,Sporting KC,Nashville SC,L 1–2,L,1,0,2,0,2022-04-09 Sporting KC vs Nashville SC
45419,2022-04-23,Sat,Regular Season,Sporting KC,Columbus Crew,D 0–0,D,0,0,0,0,2022-04-23 Sporting KC vs Columbus Crew


In [397]:
all_matches.to_csv(output/'all_matches.csv', index=False)

In [ ]:
all_players.head(2)

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Att,Cmp%,PrgP,Carries,PrgC,Att_TakeOn,Succ,Match Report,player_url,game_key
0,2024-02-24,Sat,MLS,Regular Season,Away,L 1–3,NE Revolution,D.C. United,Y,"LB,RB",...,52,75.0,2,26,0,0,0,Match Report,https://fbref.com/en/players/10d1139c/matchlog...,2024-02-24 D.C. United vs NE Revolution
1,2024-03-03,Sun,MLS,Regular Season,Home,L 0–1,NE Revolution,Toronto FC,Y,LB,...,76,85.5,6,55,3,3,1,Match Report,https://fbref.com/en/players/10d1139c/matchlog...,2024-03-03 NE Revolution vs Toronto FC


In [ ]:
all_players = all_players.apply(lambda x: x.replace("On matchday squad, but did not play", "0"))

In [ ]:
players_numeric_columns = ['Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA',
       'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att_TakeOn',
       'Succ']

for col in players_numeric_columns:
    all_players[col] = all_players[col].astype(float)

In [ ]:
final_df = pd.DataFrame()

for match in all_matches['game_key']:
    current_match_data = all_matches[all_matches['game_key'] == match].copy().reset_index()
    current_match_date = current_match_data['game_date'][0]
    current_match_date_year = int(current_match_data['game_date'][0][:4])
    home_team = current_match_data['home_team'][0]
    away_team = current_match_data['away_team'][0]
    # home_team_stats = all_players.query(f'Squad=="{home_team}" and game_key=="{match}" and Venue=="Home" and Start!="N"')[players_numeric_columns]
    # away_team_stats = all_players.query(f'Squad=="{away_team}" and game_key=="{match}" and Venue=="Away" and Start!="N"')[players_numeric_columns]
    prior_home_stats = all_players.query(f'Date<"{current_match_date}" and Date > "{current_match_date_year-1}-12-31" and Squad=="{home_team}" and Venue=="Home" and Start!="N"')
    prior_home_stats.to_csv(output/f'prior_stat_files/{current_match_date}_{home_team}_{away_team}.csv')
    prior_home_stats = prior_home_stats[players_numeric_columns]
    prior_away_stats = all_players.query(f'Date<"{current_match_date}" and Date > "{current_match_date_year-1}-12-31" and Squad=="{away_team}" and Venue=="Home" and Start!="N"')[players_numeric_columns]
    
    
    
    # home_team_stats.columns = [f'home_{x}' for x in home_team_stats.columns]
    # away_team_stats.columns = [f'away_{x}' for x in away_team_stats.columns]
    prior_home_stats.columns = [f'home_prior_{x}' for x in prior_home_stats.columns]
    prior_away_stats.columns = [f'away_prior_{x}' for x in prior_away_stats.columns]
    
    if prior_home_stats.shape[0]==0:
        temp = prior_home_stats.columns
        prior_home_stats = pd.DataFrame([0]*len(prior_home_stats.columns)).T
        prior_home_stats.columns = temp
        
    if prior_away_stats.shape[0]==0:
        temp = prior_away_stats.columns
        prior_away_stats = pd.DataFrame([0]*len(prior_away_stats.columns)).T
        prior_away_stats.columns = temp
        
    home_team_stats = pd.DataFrame(home_team_stats.describe().T['mean']).T.reset_index()
    away_team_stats = pd.DataFrame(away_team_stats.describe().T['mean']).T.reset_index()
    prior_home_stats = pd.DataFrame(prior_home_stats.describe().T['mean']).T.reset_index()
    prior_away_stats = pd.DataFrame(prior_away_stats.describe().T['mean']).T.reset_index()
    
    final_row = pd.concat([current_match_data, prior_home_stats, prior_away_stats], axis=1)
    
    final_df = pd.concat([final_df, final_row], axis=0)

,index,game_date,day,round,home_team,away_team,result,overall_result,home_score,home_penalties,away_score,away_penalties,game_key
0,1,2024-03-03,Sun,Regular Season,NE Revolution,Toronto FC,L 0–1,L,0,0,1,0,2024-03-03 NE Revolution vs Toronto FC


In [ ]:
final_df

,index,game_date,day,round,home_team,away_team,result,overall_result,home_score,home_penalties,...,away_prior_SCA,away_prior_GCA,away_prior_Cmp,away_prior_Att,away_prior_Cmp%,away_prior_PrgP,away_prior_Carries,away_prior_PrgC,away_prior_Att_TakeOn,away_prior_Succ
0,1,2024-03-03,Sun,Regular Season,NE Revolution,Toronto FC,L 0–1,L,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,3,2024-03-17,Sun,Regular Season,NE Revolution,FC Cincinnati,L 1–2,L,1,0,...,2.227273,0.000000,33.227273,41.590909,77.390909,2.500000,26.181818,1.136364,2.272727,1.045455
0,4,2024-03-23,Sat,Regular Season,NE Revolution,Chicago Fire,D 1–1,D,1,0,...,2.045455,0.318182,38.545455,47.227273,81.131818,3.681818,31.272727,1.454545,1.727273,0.590909
0,5,2024-04-06,Sat,Regular Season,NE Revolution,Charlotte,W 1–0,W,1,0,...,1.878788,0.212121,26.636364,34.818182,75.272727,2.757576,21.848485,1.363636,0.787879,0.333333
0,7,2024-05-18,Sat,Regular Season,NE Revolution,Philadelphia Union,L 0–3,L,0,0,...,2.787879,0.181818,30.893939,40.106061,75.646875,3.939394,27.106061,1.530303,0.893939,0.393939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,42808,2022-05-28,Sat,Regular Season,Sporting KC,Vancouver W'caps,L 0–1,L,0,0,...,1.651515,0.166667,27.363636,36.015152,75.268182,3.348485,23.484848,1.166667,0.893939,0.500000
0,42811,2022-07-03,Sun,Regular Season,Sporting KC,NY Red Bulls,L 0–1,L,0,0,...,1.959596,0.191919,25.606061,36.373737,69.514141,3.272727,22.616162,1.333333,1.373737,0.707071
0,43302,2022-04-09,Sat,Regular Season,Sporting KC,Nashville SC,L 1–2,L,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,43304,2022-04-23,Sat,Regular Season,Sporting KC,Columbus Crew,D 0–0,D,0,0,...,2.416667,0.187500,39.562500,48.000000,80.537500,4.354167,37.958333,2.083333,1.750000,0.854167


In [ ]:
final_df.home_team.value_counts()

home_team
LAFC                   55
FC Cincinnati          54
Philadelphia Union     53
Houston Dynamo         53
Columbus Crew          53
Seattle Sounders FC    53
Sporting KC            53
NY Red Bulls           53
Orlando City           52
Atlanta Utd            52
Nashville SC           52
NE Revolution          52
CF Montréal            52
Austin                 52
NYCFC                  52
FC Dallas              52
LA Galaxy              52
Toronto FC             51
Vancouver W'caps       51
Portland Timbers       51
Real Salt Lake         51
Colorado Rapids        51
SJ Earthquakes         51
Charlotte              51
Minnesota Utd          50
Inter Miami            50
Chicago Fire           50
D.C. United            50
St. Louis              35
Name: count, dtype: int64

In [ ]:
final_df.to_csv(output/'data_for_models.csv',index=False)

In [ ]:
current_match_data.columns

Index(['index', 'game_date', 'day', 'round', 'home_team', 'away_team',
       'result', 'overall_result', 'home_score', 'home_penalties',
       'away_score', 'away_penalties', 'game_key'],
      dtype='object')

In [ ]:
current_match_date = '2024-11-23'
current_match_date_year = 2024
home_team = "LAFC"
away_team = "Seattle Sounders FC"
current_match_data = pd.DataFrame([0, current_match_date, 'Sat', 'Conference Semifinals', home_team, away_team, '', '',0,0,0,0,'']).T
prior_home_stats = all_players.query(f'Date<"{current_match_date}" and Date > "{current_match_date_year-1}-12-31" and Squad=="{home_team}" and Venue=="Home" and Start!="N"')
prior_home_stats.to_csv(output/f'prior_stat_files/{current_match_date}_{home_team}_{away_team}.csv')
prior_home_stats = prior_home_stats[players_numeric_columns]
prior_away_stats = all_players.query(f'Date<"{current_match_date}" and Date > "{current_match_date_year-1}-12-31" and Squad=="{away_team}" and Venue=="Home" and Start!="N"')[players_numeric_columns]
prior_home_stats.columns = [f'home_prior_{x}' for x in prior_home_stats.columns]
prior_away_stats.columns = [f'away_prior_{x}' for x in prior_away_stats.columns]

if prior_home_stats.shape[0]==0:
    temp = prior_home_stats.columns
    prior_home_stats = pd.DataFrame([0]*len(prior_home_stats.columns)).T
    prior_home_stats.columns = temp
    
if prior_away_stats.shape[0]==0:
    temp = prior_away_stats.columns
    prior_away_stats = pd.DataFrame([0]*len(prior_away_stats.columns)).T
    prior_away_stats.columns = temp
    
home_team_stats = pd.DataFrame(home_team_stats.describe().T['mean']).T.reset_index()
away_team_stats = pd.DataFrame(away_team_stats.describe().T['mean']).T.reset_index()
prior_home_stats = pd.DataFrame(prior_home_stats.describe().T['mean']).T.reset_index()
prior_away_stats = pd.DataFrame(prior_away_stats.describe().T['mean']).T.reset_index()

pred_row = pd.concat([current_match_data, prior_home_stats, prior_away_stats], axis=1)

In [ ]:
pred_row

,0,1,2,3,4,5,6,7,8,9,...,away_prior_SCA,away_prior_GCA,away_prior_Cmp,away_prior_Att,away_prior_Cmp%,away_prior_PrgP,away_prior_Carries,away_prior_PrgC,away_prior_Att_TakeOn,away_prior_Succ
0,0,2024-11-23,Sat,Conference Semifinals,LAFC,Seattle Sounders FC,,,0,0,...,2.190751,0.202312,39.323699,47.231214,81.106395,3.867052,32.213873,1.398844,1.242775,0.560694


In [ ]:
pred_row.to_csv('test6.csv', index=False)